In [37]:
from rdflib import Graph, Namespace
from pyvis.network import Network
from IPython.display import IFrame, display

# 1. Adjust the path to your TTL file
TTL_FILE = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"

# 2. Create and parse the graph
g = Graph()
g.parse(TTL_FILE, format="turtle")

# 3. Define namespaces
ILCD = Namespace("https://example.org/ilcd/")
DIN  = Namespace("https://example.org/din276/")
CC   = Namespace("https://example.org/concreteclass/")
OBD  = Namespace("https://example.org/obd/")
BKI  = Namespace("https://example.org/bki/")

# 4. Find a subset of EPDs
EPD_LIMIT = 2
q_epds = f"""
SELECT ?epd
WHERE {{
  ?epd a <{ILCD}ProcessDataSet> .
}}
LIMIT {EPD_LIMIT}
"""
results_epd = g.query(q_epds)
epd_uris = [str(row.epd) for row in results_epd]

# ---------------------------------------------------------------------------------------
# Create the PyVis network
# ---------------------------------------------------------------------------------------
net = Network(
    notebook=True,
    height="1080px",
    width="1920px",
    directed=True,
    cdn_resources="in_line",
)


# Force Atlas 2
net.force_atlas_2based(
    central_gravity=0.1,
    spring_length=2000,
    overlap=1
)

# Disable physics
net.toggle_physics(False)

# Show interactive physics controls
# net.show_buttons(filter_=['physics'])


epd_number = 0

for epd_uri in epd_uris:
    epd_number += 1
    print(f"Processing EPD {epd_number}: {epd_uri}")
    
    # ------------------------------------------------------------------
    # LEVEL 0: EPD node
    # ------------------------------------------------------------------
    net.add_node(
        epd_uri, 
        label=f"Instance {epd_number}", 
        shape="ellipse", 
        color="#a2d2ff",
        level=0
    )

    # ------------------------------------------------------------------
    # 1) DIN 276 cost groups: only 322, 331
    # ------------------------------------------------------------------
    q_din = f"""
    SELECT ?costgroup ?cgNotation
    WHERE {{
      <{epd_uri}> <{DIN}hasDIN276CostGroup> ?costgroup .
      ?costgroup  skos:notation ?cgNotation .
    }}
    """
    results_din = g.query(q_din)
    for row in results_din:
        print(row)
        cg_uri = str(row.costgroup)
        cg_notation = str(row.cgNotation)
        cg_label = cg_notation.split("/")[-1]
        # Skip if not cost group 322 or cost group 331
        if cg_label not in ["322", "331"]:
            continue

        # We'll place cost groups at level=1
        net.add_node(
            cg_uri, 
            label=cg_label, 
            shape="box", 
            # color="#fef9c3", 
            color="#f5f5dc", 
            level=1
        )
        net.add_edge(epd_uri, cg_uri, label="hasDIN276CostGroup")

        # If costgroup_331, include BKI elements referencing it
        if cg_label == "322":
            q_bki = f"""
            SELECT ?bkiElem ?bkiName
            WHERE {{
              bki:element_291c4af9512d41538ca62691929c1d0f a <{BKI}BKIElement> ;
                       <{BKI}name> ?bkiName ;
                       <{DIN}hasDIN276CostGroup> <{cg_uri}> .
            }}
            """
            bki_results = g.query(q_bki)
            for bki_row in bki_results:
                bki_elem  = str(bki_row.bkiElem)
                bki_label = str(bki_row.bkiName)
                
                # Put BKI elements at level=2
                net.add_node(
                    bki_elem, 
                    label=bki_label, 
                    shape="box", 
                    color="#d9f7be",
                    level=4
                )
                net.add_edge(cg_uri, bki_elem, label="hasBKIElement")

    # ------------------------------------------------------------------
    # 2) Strength / Weight Classification
    # ------------------------------------------------------------------
    q_strength = f"""
    SELECT ?strength
    WHERE {{
      <{epd_uri}> <{CC}hasStrengthClassification> ?strength .
    }}
    """
    strength_results = g.query(q_strength)
    for srow in strength_results:
        strength_uri = str(srow.strength)
        s_label = strength_uri.split("/")[-1]
        net.add_node(
            strength_uri, 
            label=s_label, 
            shape="box", 
            color="#fde2e4",
            level=3
        )
        net.add_edge(epd_uri, strength_uri, label="hasStrengthClassification")

    q_weight = f"""
    SELECT ?weight
    WHERE {{
      <{epd_uri}> <{CC}hasWeightClassification> ?weight .
    }}
    """
    weight_results = g.query(q_weight)
    for wrow in weight_results:
        weight_uri = str(wrow.weight)
        w_label = weight_uri.split("/")[-1]
        net.add_node(
            weight_uri, 
            label=w_label, 
            shape="box", 
            color="#fde2e4",
            level=3
        )
        net.add_edge(epd_uri, weight_uri, label="hasWeightClassification")

    # ------------------------------------------------------------------
    # 3) classificationInformation => level=1
    # ------------------------------------------------------------------
    q_classInfo = f"""
    SELECT ?classInfo
    WHERE {{
      <{epd_uri}> <{ILCD}processInformation> ?procInfo .
      ?procInfo <{ILCD}dataSetInformation> ?dataSetInfo .
      ?dataSetInfo <{ILCD}classificationInformation> ?classInfo .
    }}
    """
    cinfo_results = g.query(q_classInfo)

    for ci_row in cinfo_results:
        ci_uri = str(ci_row.classInfo)
        net.add_node(
            ci_uri, 
            label="classificationInformation", 
            shape="ellipse", 
            color="#e7e7e7",
            level=1
        )
        net.add_edge(epd_uri, ci_uri, label="hasClassificationInfo")

        # Classification nodes => level=2
        q_classifications = f"""
        SELECT ?classification ?className
        WHERE {{
          <{ci_uri}> <{ILCD}classification> ?classification .
          OPTIONAL {{ ?classification <{ILCD}name> ?className . }}
        }}
        """
        sub_results = g.query(q_classifications)
        for sub_row in sub_results:
            cls_uri = str(sub_row.classification)
            cls_name = sub_row.className if sub_row.className else "Classification"

            net.add_node(
                cls_uri, 
                label=cls_name, 
                shape="box", 
                color="#ccccee",
                level=2
            )
            net.add_edge(ci_uri, cls_uri, label="hasClassification")

            # Decide how to handle entries
            skip_entries = False
            keep_only_rc_beton = False
            lower_cls = cls_name.lower()
            if "epdnorge" in lower_cls or "ibucategories" in lower_cls:
                skip_entries = True
            elif "oekobau" in lower_cls:
                keep_only_rc_beton = True

            if skip_entries:
                continue

            # classification entries => level=3
            q_entries = f"""
            SELECT ?entry ?classVal ?canonCat
            WHERE {{
              <{cls_uri}> <{ILCD}classEntries> ?entry .
              ?entry <{ILCD}value> ?classVal .
              OPTIONAL {{ ?entry <{OBD}hasCanonicalCategory> ?canonCat . }}
            }}
            """
            entry_results = g.query(q_entries)
            for e_row in entry_results:
                entry_uri = str(e_row.entry)
                entry_val = str(e_row.classVal)

                if keep_only_rc_beton:
                    # Only keep "Ready mixed concrete" or "Beton"
                    if entry_val not in ["Ready mixed concrete", "Beton"]:
                        continue

                net.add_node(
                    entry_uri, 
                    label=entry_val, 
                    shape="ellipse", 
                    color="#ffffcc",
                    level=3
                )
                net.add_edge(cls_uri, entry_uri, label="hasMaterialCategory")

                # canonical category => level=4
                if e_row.canonCat:
                    canon_uri = str(e_row.canonCat)
                    # fetch SKOS label
                    q_canonLabel = f"""
                    SELECT ?pref
                    WHERE {{
                      <{canon_uri}> <http://www.w3.org/2004/02/skos/core#prefLabel> ?pref .
                      FILTER(lang(?pref) = "en")
                    }}
                    LIMIT 1
                    """
                    canon_label_res = g.query(q_canonLabel)
                    canon_label = None
                    for clrow in canon_label_res:
                        canon_label = str(clrow.pref)
                    if not canon_label:
                        canon_label = canon_uri.split("/")[-1]

                    net.add_node(
                        canon_uri, 
                        label=canon_label, 
                        shape="ellipse", 
                        color="#ffeedb",
                        level=4
                    )
                    net.add_edge(entry_uri, canon_uri, label="hasCanonicalCategory")

# ---------------------------------------------------------------------------------------
# Render & display
# ---------------------------------------------------------------------------------------
html_file = "knowledge_graph.html"
net.show(html_file)
display(IFrame(html_file, width="100%", height="100%"))


Processing EPD 1: https://example.org/ilcd/01145859af704f3096abf4a8f3e92f9d
(rdflib.term.URIRef('https://example.org/din276/costgroup_320'), rdflib.term.Literal('320'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_322'), rdflib.term.Literal('322'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_350'), rdflib.term.Literal('350'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_351'), rdflib.term.Literal('351'))
Processing EPD 2: https://example.org/ilcd/0643c04910464d518ddc760f32642a72
(rdflib.term.URIRef('https://example.org/din276/costgroup_320'), rdflib.term.Literal('320'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_322'), rdflib.term.Literal('322'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_330'), rdflib.term.Literal('330'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_331'), rdflib.term.Literal('331'))
(rdflib.term.URIRef('https://example.org/din276/costgroup_340'), rdflib.term.Literal('340'))
(rdflib.ter

# WP3 Figure: ÖKOBAUDAT Categories

In [42]:
from rdflib import Graph, Namespace, RDF, Literal
from rdflib.namespace import SKOS
from pyvis.network import Network
from IPython.display import IFrame, display
import json, os, pathlib

# Define configuration values
TTL_FILE    = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"
TARGET_UUID = "0643c04910464d518ddc760f32642a72"
LAYOUT_FILE = "layout_WP3_3-2_02.json"
HTML_OUT    = "category_graph_single.html"
RANDOM_SEED = 1

# Load RDF graph from Turtle
g = Graph().parse(TTL_FILE, format="turtle")
ILCD = Namespace("https://example.org/ilcd/")
OBD  = Namespace("https://example.org/obd/")
dataset_uri = ILCD[TARGET_UUID]

# Select ÖKOBAUDAT classification
q_cls = f"""
SELECT ?cls WHERE {{
  <{dataset_uri}> <{ILCD}processInformation> ?p .
  ?p <{ILCD}dataSetInformation> ?dsi .
  ?dsi <{ILCD}classificationInformation> ?ci .
  ?ci <{ILCD}classification> ?cls .
  ?cls <{ILCD}name> "OEKOBAU.DAT"
}}
"""
cls_uri = next(iter(g.query(q_cls))).cls

# Provide helper for readable labels
def best_label(node):
    lits = list(g.objects(node, SKOS.prefLabel))
    for lang in ("en", "de"):
        for lit in lits:
            if isinstance(lit, Literal) and lit.language == lang:
                return str(lit)
    return str(lits[0]) if lits else str(node).split("/")[-1]

# Read saved coordinates if the file exists
coords = json.load(open(LAYOUT_FILE)) if os.path.isfile(LAYOUT_FILE) else {}

# Create PyVis network with physics disabled
net = Network(notebook=True, height="1080px", width="1920px",
              directed=True, cdn_resources="in_line")

net.set_options(f'{{"layout":{{"randomSeed":{RANDOM_SEED}}},'
                f'"physics":{{"enabled":false}}}}')

# Define helper to add nodes once
added = set()
def add_node(uri, label, shape, color, level, font_size=14):
    if uri in added:
        return
    net.add_node(uri, label=label, shape=shape, color=color,
                 level=level, font={"size": font_size},
                 **coords.get(uri, {}))
    added.add(uri)

# Add classification, entries, and canonical categories
add_node(str(cls_uri), "ÖKOBAUDAT", "box", "#ccccee", 0)
for entry in g.objects(cls_uri, ILCD.classEntries):
    lbl  = str(g.value(entry, ILCD.value) or entry.split("/")[-1])
    lvl  = int(g.value(entry, ILCD.level) or 0) + 1
    add_node(str(entry), lbl, "ellipse", "#ffffcc", lvl)
    net.add_edge(str(cls_uri), str(entry), label="classEntry")

    canon = g.value(entry, OBD.hasCanonicalCategory)
    if canon:
        add_node(str(canon), best_label(canon), "ellipse", "#ffeedb", lvl + 1)
        net.add_edge(str(entry), str(canon), label="hasCanonicalCategory")

# Add SKOS concepts and hierarchy edges
scheme = OBD.OEKOBAU_DAT
for cat in g.subjects(RDF.type, SKOS.Concept):
    if (cat, SKOS.inScheme, scheme) not in g:
        continue
    add_node(str(cat), best_label(cat), "ellipse", "#ffeedb", 2)

    for nar in g.objects(cat, SKOS.narrower):
        if (nar, SKOS.inScheme, scheme) in g:
            net.add_edge(str(cat), str(nar), label="narrower",
                         smooth={"type": "curvedCW", "roundness": 0.2})

    for bro in g.objects(cat, SKOS.broader):
        if (bro, SKOS.inScheme, scheme) in g:
            net.add_edge(str(cat), str(bro), label="broader", dashes=True,
                         color="#9f9f80",
                         smooth={"type": "curvedCCW", "roundness": 0.05})

# Add legend nodes with small font and allow dragging
legend = [
    ("ilcd:Classification"     , "box"    , "#ccccee"),
    ("ilcd:ClassificationEntry", "ellipse", "#ffffcc"),
    ("obd/skos:Concept"        , "ellipse", "#ffeedb")
]
for i, (lab, shape, col) in enumerate(legend):
    add_node(f"legend_{i}", lab, shape, col, 0, font_size=10)

# Inject buttons
extra_js = f"""
<script>
window.addEventListener('load', () => {{

  /*  Save layout  -------------------------------------------------------- */
  const btnSave = Object.assign(document.createElement('button'), {{
    innerText:'💾 Save layout',
    style:'position:absolute;left:10px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnSave);
  btnSave.onclick = () => {{
    const blob = new Blob(
      [JSON.stringify(network.getPositions(), null, 2)],
      {{type:'application/json'}});
    const url = URL.createObjectURL(blob);
    Object.assign(document.createElement('a'), {{
      href:url, download:'{LAYOUT_FILE}'
    }}).click();
    URL.revokeObjectURL(url);
  }};

  /*  Export PNG  --------------------------------------------------------- */
  const btnPng = Object.assign(document.createElement('button'), {{
    innerText:'📷 Export PNG',
    style:'position:absolute;left:140px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnPng);
  btnPng.onclick = () => {{
    const canvas  = network.canvas.frame.canvas;          // grab the canvas
    const pngData = canvas.toDataURL('image/png');        // → data URL
    const link    = Object.assign(document.createElement('a'), {{
      href: pngData, download:'graph.png'
    }});
    link.click();
  }};
}});
</script>
"""

# Write HTML file and display it
html = net.generate_html().replace("</body>", extra_js + "\\n</body>")
pathlib.Path(HTML_OUT).write_text(html, encoding="utf-8")
display(IFrame(HTML_OUT, width="100%", height="100%"))


# WP3 Figure: DIN 276 Cost Groups

In [44]:
from rdflib import Graph, Namespace, RDF
from rdflib.namespace import SKOS
from pyvis.network import Network
from IPython.display import IFrame, display
import json, os, pathlib

# Configuration
TTL_FILE    = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"
LAYOUT_FILE = "layout_WP3_3-3.json"
HTML_OUT    = "din276_costgroups.html"
RANDOM_SEED = 2

# Namespaces
DIN  = Namespace("https://example.org/din276/")
ILCD = Namespace("https://example.org/ilcd/")

# Load RDF graph
g = Graph().parse(TTL_FILE, format="turtle")

# Load saved node coordinates if available
coords = json.load(open(LAYOUT_FILE)) if os.path.isfile(LAYOUT_FILE) else {}

# Create PyVis network
net = Network(notebook=True, height="1080px", width="1920px",
              directed=True, cdn_resources="in_line")
net.set_options(f'{{"layout":{{"randomSeed":{RANDOM_SEED}}},'
                f'"physics":{{"enabled":false}}}}')

added = set()
def add_node(uri, label, shape, color, level, font_size=14):
    if uri in added:
        return
    net.add_node(uri, label=label, shape=shape, color=color,
                 level=level, font={"size": font_size},
                 **coords.get(uri, {}))
    added.add(uri)

# Select the first two ProcessDataSet instances with DIN276 cost groups
datasets = [
    ds for ds in g.subjects(RDF.type, ILCD.ProcessDataSet)
    if (ds, DIN.hasDIN276CostGroup, None) in g
][:2]
ds1, ds2 = datasets

# Gather cost groups for each
cgs1 = list(g.objects(ds1, DIN.hasDIN276CostGroup))
cgs2 = list(g.objects(ds2, DIN.hasDIN276CostGroup))

# Determine a shared cost group (if any), or fallback to the first of ds1
shared = next((cg for cg in cgs1 if cg in cgs2), cgs1[0])

# Pick one additional from each (distinct from shared)
other1 = next((cg for cg in cgs1 if cg != shared), shared)
other2 = next((cg for cg in cgs2 if cg != shared), shared)

# Build list of (dataset, costgroup) pairs—two per instance
pairs = [
    (ds1, shared),
    (ds1, other1),
    (ds2, shared),
    (ds2, other2),
]

# Add dataset nodes labeled "Instance 1", "Instance 2"
add_node(str(ds1), "Instance 1", "box", "#ccccee", 0)
add_node(str(ds2), "Instance 2", "box", "#ccccee", 0)

# Add cost‐group nodes (label = notation) and edges
for ds, cg in pairs:
    ds_uri = str(ds)
    cg_uri = str(cg)
    notation = g.value(cg, SKOS.notation) or ""
    add_node(cg_uri, str(notation), "ellipse", "#f5f5dc", 1)
    net.add_edge(ds_uri, cg_uri, label="hasDIN276CostGroup")

# Add SKOS broader/narrower links among the shown cost groups
for _, cg in pairs:
    cg_uri = str(cg)
    for nar in g.objects(cg, SKOS.narrower):
        nar_uri = str(nar)
        if nar_uri in added:
            net.add_edge(cg_uri, nar_uri, label="narrower",
                         smooth={"type": "curvedCW", "roundness": 0.2})
    for bro in g.objects(cg, SKOS.broader):
        bro_uri = str(bro)
        if bro_uri in added:
            net.add_edge(cg_uri, bro_uri, label="broader", dashes=True,
                         color="#9f9f80",
                         smooth={"type": "curvedCCW", "roundness": 0.05})

# Legend
legend = [
    ("ilcd:ProcessDataSet", "box",     "#ccccee"),
    ("din/skos:Concept", "ellipse", "#f5f5dc")
]
for i, (lab, shape, col) in enumerate(legend):
    add_node(f"legend_{i}", lab, shape, col, 2, font_size=10)

# Inject buttons
extra_js = f"""
<script>
window.addEventListener('load', () => {{

  /*  Save layout  -------------------------------------------------------- */
  const btnSave = Object.assign(document.createElement('button'), {{
    innerText:'💾 Save layout',
    style:'position:absolute;left:10px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnSave);
  btnSave.onclick = () => {{
    const blob = new Blob(
      [JSON.stringify(network.getPositions(), null, 2)],
      {{type:'application/json'}});
    const url = URL.createObjectURL(blob);
    Object.assign(document.createElement('a'), {{
      href:url, download:'{LAYOUT_FILE}'
    }}).click();
    URL.revokeObjectURL(url);
  }};

  /*  Export PNG  --------------------------------------------------------- */
  const btnPng = Object.assign(document.createElement('button'), {{
    innerText:'📷 Export PNG',
    style:'position:absolute;left:140px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnPng);
  btnPng.onclick = () => {{
    const canvas  = network.canvas.frame.canvas;          // grab the canvas
    const pngData = canvas.toDataURL('image/png');        // → data URL
    const link    = Object.assign(document.createElement('a'), {{
      href: pngData, download:'graph.png'
    }});
    link.click();
  }};

}});
</script>
"""

# Generate HTML and display
html = net.generate_html().replace("</body>", extra_js + "\n</body>")
pathlib.Path(HTML_OUT).write_text(html, encoding="utf-8")
display(IFrame(HTML_OUT, width="100%", height="800px"))


In [45]:
# --------------------------------------------------------------------------
#  BKI Element ⇄ DIN 276 cost‑group  +  Layers  —  interactive graph
# --------------------------------------------------------------------------
from rdflib import Graph, Namespace, RDF
from rdflib.namespace import SKOS
from pyvis.network import Network
from IPython.display import IFrame, display
import json, os, pathlib

# ── Configuration ─────────────────────────────────────────────────────────
TTL_FILE    = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"            # path to your RDF
LAYOUT_FILE = "layout_bki_din.json"   # stores / restores dragged positions
HTML_OUT    = "bki_din_layers.html"   # output file
RANDOM_SEED = 3                       # fixed seed ⇒ deterministic layout

# ── Namespaces ────────────────────────────────────────────────────────────
BKI = Namespace("https://example.org/bki/")
DIN = Namespace("https://example.org/din276/")

# ── Load RDF graph ────────────────────────────────────────────────────────
g = Graph().parse(TTL_FILE, format="turtle")

# ── Read saved coordinates (if any) ───────────────────────────────────────
coords = json.load(open(LAYOUT_FILE)) if os.path.isfile(LAYOUT_FILE) else {}

# ── Create PyVis network ──────────────────────────────────────────────────
net = Network(notebook=True, height="1080px", width="1920px",
              directed=True, cdn_resources="in_line")
net.set_options(f'{{"layout":{{"randomSeed":{RANDOM_SEED}}},'
                f'"physics":{{"enabled":false}}}}')

added = set()
def add_node(uri, label, shape, color, level, font_size=14):
    """Add node once, reusing any saved x/y coords."""
    if uri in added:
        return
    net.add_node(uri, label=label, shape=shape, color=color,
                 level=level, font={"size": font_size},
                 **coords.get(uri, {}))
    added.add(uri)

# ── Build the view ────────────────────────────────────────────────────────
for elem in g.subjects(RDF.type, BKI.BKIElement):
    elem_uri  = str(elem)
    elem_name = g.value(elem, BKI.name) or elem_uri.split("/")[-1]
    add_node(elem_uri, str(elem_name), "box", "#ccccee", 0)

    # DIN 276 cost groups
    for cg in g.objects(elem, DIN.hasDIN276CostGroup):
        cg_uri  = str(cg)
        cg_lab  = g.value(cg, SKOS.notation) or cg_uri.split("/")[-1]
        add_node(cg_uri, str(cg_lab), "ellipse", "#ffffcc", 1)
        net.add_edge(elem_uri, cg_uri, label="hasDIN276CostGroup")

    # Layers
    for layer in g.objects(elem, BKI.hasLayer):
        lay_uri = str(layer)
        lay_lab = g.value(layer, BKI.processConfigName) or lay_uri.split("/")[-1]
        add_node(lay_uri, str(lay_lab), "ellipse", "#ffeedb", 1)
        net.add_edge(elem_uri, lay_uri, label="hasLayer")

# ── Legend ────────────────────────────────────────────────────────────────
legend = [
    ("bki:BKIElement" , "box"    , "#ccccee"),
    ("din276:CostGroup", "ellipse", "#ffffcc"),
    ("bki:Layer"      , "ellipse", "#ffeedb")
]
for i, (lab, shape, col) in enumerate(legend):
    add_node(f"legend_{i}", lab, shape, col, 2, font_size=10)

# ── “Save layout” / “Export PNG” buttons ──────────────────────────────────
extra_js = f"""
<script>
window.addEventListener('load', () => {{

  /*  Save layout --------------------------------------------------------- */
  const btnSave = Object.assign(document.createElement('button'), {{
    innerText:'💾 Save layout',
    style:'position:absolute;left:10px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnSave);
  btnSave.onclick = () => {{
    const blob = new Blob(
      [JSON.stringify(network.getPositions(), null, 2)],
      {{type:'application/json'}});
    const url = URL.createObjectURL(blob);
    Object.assign(document.createElement('a'), {{
      href:url, download:'{LAYOUT_FILE}'
    }}).click();
    URL.revokeObjectURL(url);
  }};

  /*  Export PNG ---------------------------------------------------------- */
  const btnPng = Object.assign(document.createElement('button'), {{
    innerText:'📷 Export PNG',
    style:'position:absolute;left:140px;top:10px;z-index:999;'
  }});
  document.body.appendChild(btnPng);
  btnPng.onclick = () => {{
    const canvas  = network.canvas.frame.canvas;
    const pngData = canvas.toDataURL('image/png');
    Object.assign(document.createElement('a'), {{
      href: pngData,
      download:'bki_din_layers.png'
    }}).click();
  }};
}});
</script>
"""

# ── Generate HTML and display ─────────────────────────────────────────────
html = net.generate_html().replace("</body>", extra_js + "\n</body>")
pathlib.Path(HTML_OUT).write_text(html, encoding="utf-8")
display(IFrame(HTML_OUT, width="100%", height="800px"))
